# Sea-Bird Scientific Python Toolkit - CTD Processing
## Introduction
This python notebook has been created to give Sea-Bird customers and the scientific community at large a set of python code to be used in the processing and analysis of data from the CTD family of SBS instruments. This toolkit was developed in conjunction with the Orca project[<sup>1</sup>](#fn1). Great care has been taken to make sure that the python code that makes up this toolkit and its supporting packages produces the same results as the Orca application.

Please see the [Installation Instructions<sup>2</sup>](#fn2) for assitance in getting your environment set up to use the toolkit.

Please reach out to [Sea-Bird customer support](https://www.seabird.com/support) for assistance or to request additional features.

<span id="fn1"> <sup>1</sup> TODO: change Orca to the final app name<br>
<span id="fn2"> <sup>2</sup> TODO: add link to setup instructions to be created under [NSI-2763](https://jira.sea-birdscientific.com/browse/NSI-2763)

## Python Data Processing and Visualization
- [Data Conversion](#Data-Conversion)
- [Low Pass Filter](#Low-Pass-Filter)
- [Align CTD](#Align-CTD)
- [Cell Thermal Mass](#Cell-Thermal-Mass)
- [Loop Edit](#Loop-Edit)
- [Derive TEOS-10](#Derive-TEOS-10)
- [Wild Edit](#Wild-Edit)
- [Window Filter](#Window-Filter)


## Init

The following initialization code section is used to import required libraries and to create a helper function that is used for plotting scatter charts with the Plotly package.

In [ ]:
# Native imports

# Third-party imports
import gsw
import numpy as np
import pandas as pd
import os.path

# Sea-Bird imports
import sbs.process.cal_coefficients as cc
import sbs.process.conversion as c
import sbs.process.processing as p
import sbs.process.instrument_data as id
import sbs.visualize.visualization as v

def plot(data, title, x_names, x_bounds={ 0: [17, 24], 1: [-1, 6], 2: [-1, 6], 3: [-1, 6] }):
    config = v.ChartConfig(
        title = title,
        x_names = x_names,
        y_names = ['press'],
        z_names = [],
        chart_type = "overlay",
        bounds={
            'x': x_bounds,
            # 'y': { 0: [29, 32], 1: [29, 32], 2: [29, 32], 3: [29, 32] }
            },
        plot_loop_edit_flags=False,
        lift_pen_over_bad_data=True
    )

    chart_data = v.ChartData(data, config)
    fig = v.plot_xy_chart(chart_data, config)
    fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_layout(height=800)
    fig.show()


## Data Conversion

This section shows how to convert raw data contained in a .hex file into engineering units for the following instruments:  
- SBE37SM
- SBE37SMP
- SBE37SMPODO
- SBE37IM
- SBE37IMP
- SBE37IMPODO
- SBE19Plus

Supported output units include temperature, conductivity, pressure, density, potential density, and depth.

In [ ]:
hex_file = os.path.join('.', 'example_data','E8001.hex')
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.SBE19Plus,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.ExtVolt0,
        id.Sensors.ExtVolt1,
        id.Sensors.ExtVolt2,
        id.Sensors.ExtVolt3,
    ]
)
cal = cc.SN6130 # fake some cal coefficients for demonstration

temperature = c.convert_temperature_array(
    temperature_counts=raw_data['temperature'].values,
    a0=cal.A0,
    a1=cal.A1,
    a2=cal.A2,
    a3=cal.A3,
    ITS90=False,
    celsius=True,
    use_MV_R=True, # TODO: clearly define MV and R
)

pressure = c.convert_pressure_array(
    raw_data['pressure'].values,
    raw_data['temperature compensation'],
    False,
    cal.PA0,
    cal.PA1,
    cal.PA2,
    cal.PTEMPA0,
    cal.PTEMPA1,
    cal.PTEMPA2,
    cal.PTCA0,
    cal.PTCA1,
    cal.PTCA2,
    cal.PTCB0,
    cal.PTCB1,
    cal.PTCB2
)

conductivity = c.convert_conductivity_array(
    raw_data['conductivity'].values,
    temperature,
    pressure,
    cal.G,
    cal.H,
    cal.I,
    cal.J,
    cal.CPCOR,
    cal.CTCOR,
    cal.WBOTC
)

flag = np.zeros(len(temperature))

tcp_data = np.array([temperature, pressure, conductivity, flag]).transpose()
data = pd.DataFrame(data=tcp_data, columns=['temp', 'press', 'cond', 'flag'])

plot(
    data=data,
    title='Data Conversion',
    x_names=['temp', 'cond'],
    x_bounds={}
)

## Low Pass Filter

This section shows how to run a low-pass filter on one or more columns of converted data. A low-pass filter smoothes high frequency data (data that changes quickly). To make zero phase (no time shift), the filter is first run forward through the data and then run backward through the forward-filtered data. This removes any delays caused by the filter.  

Pressure data is typically filtered with a time constant equal to four times the CTD scan rate. Conductivity and temperature are typically filtered for *some* CTDs. Typical time constants are:  

| Instrument              | Temperature | Conductivity | Pressure |
|-------------------------|-------------|--------------|----------|
| SBE 19plus or 19plus V2 | 0.5         | 0.5          | 1.0      |

### Filter Formula
For a low-pass filter with time constant $Γ$:  
&emsp; $Γ= 1/2πf$  
&emsp; $T =$ sample interval (seconds)  
&emsp; $S_0 = 1/Γ$  
  
Laplace transform of the transfer function of a low-pass filter (single pole) with a time constant of $Γ$ seconds is:  

&emsp; $H(s) = \frac{1}{1 + (S/S_0)}$  
  
Using the bilinear transform:  

&emsp; $S - f(z) \overset{\Delta}{=} \frac{2(1-z^{-1})}{T(1+z^{-1})} = \frac{2(z-1)}{T(z+1)}$  

&emsp; $H(z) = \frac{1}{1 + \frac{2(z-1)}{T(z+1)S_0}} = \frac{z^{-1}+1}{1 + \frac{2}{TS_0}(1+\frac{1-2/TS_0}{1+2/TS_0}z^{-1})}$  

If:  

&emsp; $A = \frac{1}{1 + \frac{2}{TS_0}}$  

&emsp; $B = \frac{1 - \frac{2}{TS_0}}{1 + \frac{2}{TS_0}}$  

Then:  

&emsp; $H(z) = \frac{Y(z)}{X(z)} = \frac{A(z^{-1}+1)}{1+Bz^{-1}}$  
  
Where $z^{-1}$ is the unit delay (one scan behind).  
&emsp; $Y(z) (1 + Bz -1 ) = X(z) A (z -1 + 1)$  
&emsp; $y[N] + By[N-1] = Ax[N-1] + Ax[N]$  
&emsp; $y[N] = A(x[N] + x[N-1]) - By[N-1]$  

In [ ]:
data['filtered_temp'] = p.low_pass_filter(
    x=data['temp'],
    time_constant=1,
    sample_interval=0.25
)

data['filtered_cond'] = p.low_pass_filter(
    x=data['cond'],
    time_constant=1,
    sample_interval=0.25
)

plot(
    data=data,
    title='Low Pass Filter',
    x_names=['temp', 'cond', 'filtered_cond'],
    x_bounds={}
)

## Align CTD
This section shows how the Align CTD function aligns parameter data in time, relative to pressure. This ensures that calculations of salinity, dissolved oxygen concentration, and other parameters are made with measurements from the same parcel of water. Typically, Align CTD is used to align temperature, conductivity, and oxygen measurements relative to pressure.  

There are three principal causes of misalignment of CTD measurements:
- physical misalignment of the sensors in depth
- inherent time delay (time constants) of the sensor responses
- water transit time delay in the pumped plumbing line - the time it takes the parcel of water to go through the plumbing to each sensor (or, for free-flushing sensors, the corresponding flushing delay, which depends on profiling speed)  

When measurements are correctly aligned, salinity spiking (and density) errors are minimized, and oxygen data corresponds to the proper pressure (e.g., temperature vs. oxygen plots agree between down and up profiles).  

### Conductivity and Temperature

Temperature and conductivity are often misaligned with respect to pressure. Shifting temperature and conductivity relative to pressure can compensate.  

The best diagnostic of correct alignment is the removal of salinity spikes that coincide with very sharp temperature steps. To determine the best alignment, make a plot 10 meters of temperature and salinity data at a depth that contains a very sharp temperature step. For the downcast, when temperature and salinity decrease with increased pressure:  
- A negative salinity spike at the conductivity step means that conductivity leads temperature (conductivity sensor sees step before temperature sensor does). Advance conductivity relative to temperature a negative number of seconds.  
- Conversely, if the salinity spike is positive, advance conductivity relative to temperature a positive number of seconds.  

The best alignment of conductivity with respect to temperature is obtained when the salinity spikes are minimized. Some experimentation with different advances is required to find the best alignment.  

#### Typical Temperature Alignment
The SBE 19, 19plus, and 19plus V2 use a temperature sensor with a relatively slow response time.

| Instrument          | Advance of temperature relative to pressure |
|---------------------|---------------------------------------------|
| 19plus or 19plus V2 | +0.5 seconds                                |

#### Typical Conductivity Alignment
For an SBE 19plus or 19plus V2 with a standard 2000-rpm pump, do not advance conductivity.  
However, if temperature is advanced relative to pressure and you do not want to change the relative timing of temperature and conductivity, you must add the same advance to conductivity.  

### Oxygen
Oxygen data is also systematically delayed with respect to pressure. The two primary causes are the long time constant of the oxygen sensor (for the SBE 43, ranging from 2 seconds at 25 ºC to approximately 5 seconds at 0 ºC) and an additional delay from the transit time of water in the pumped plumbing line. As with temperature and conductivity, you can compensate for this delay by shifting oxygen data relative to pressure.  

| Instrument          | Advance of oxygen relative to pressure |
|---------------------|----------------------------------------|
| 19plus or 19plus V2 | +3 to +7 seconds                       |

In [ ]:
data['aligned_temp_05'] = p.align_ctd(
    x=data['filtered_temp'],
    offset=0.5,
    sample_interval=0.25
)

data['aligned_temp_10'] = p.align_ctd(
    x=data['filtered_temp'],
    offset=10,
    sample_interval=0.25
)

plot(
    data=data,
    title='ALign CTD',
    x_names=['filtered_temp', 'aligned_temp_05', 'aligned_temp_10'],
    x_bounds={}
)

## Cell Thermal Mass
This section shows the use of the Cell Thermal Mass function which uses a recursive filter to remove conductivity cell thermal mass effects from the measured conductivity. Typical values for alpha and 1/beta are:  
  
| Instrument                                             | alpha | 1/beta |
|--------------------------------------------------------|-------|--------|
| SBE 19plus or 19plus V2 with TC duct and 2000 rpm pump | 0.04  | 8.0    |

### Cell Thermal Mass Formulas

&emsp; $a = \frac{2*alpha}{(sample\ interval * beta + 2)}$  

&emsp; $b = 1 - \frac{2a}{alpha}$  

&emsp; $\frac{c}{dT} = 0.1 + 6e\text{-}4*(temperature-20)$  

&emsp; $dT = temperature - previous\ temperature$  

&emsp; $ctm = -b*previous\ ctm+a*\frac{c}{dT}*dT$  

Where sample interval is in seconds, temperature is in °C, and ctm is in Sm.  

To determine the values for alpha and beta, see:  
Lueck, R.G., 1990: Thermal Inertia of Conductivity Cells: Theory., American Meteorological Society Oct 1990, 741-755.  

In [ ]:
data['corrected_cond'] = p.cell_thermal_mass(
    temperature_C=data['filtered_temp'],
    conductivity_Sm=data['filtered_cond'],
    amplitude=0.03,
    time_constant=7,
    sample_interval=0.25
)

plot(
    data=data,
    title='Cell Thermal Mass',
    x_names=['temp', 'cond', 'filtered_cond', 'corrected_cond'],
    x_bounds={}
)

## Loop Edit
This section shows how the Loop Edit function identifies scans as bad by setting the flag value associated with the scan to badflag in input .cnv files that have pressure slowdowns or reversals (typically caused by ship heave). Optionally, Loop Edit can also identify scans associated with an initial surface soak with badflag. The badflag value defaults to -9.99e-29.

Loop Edit operates on three successive scans to determine velocity. This is such a fine scale that noise in the pressure channel from counting jitter or other unknown sources can cause Loop Edit to mark scans with badflag in error. Therefore, you must run Filter on the pressure data to reduce noise before you run Loop Edit. See Filter for pressure filter recommendations for each instrument.  

In [ ]:
p.loop_edit_pressure(
    pressure=data['press'].values,
    latitude=0,
    flag=data['flag'].values,
    sample_interval=0.25,
    min_velocity_type=p.MinVelocityType.FIXED,
    min_velocity=0.1,
    window_size=3,
    mean_speed_percent=20,
    remove_surface_soak=True,
    min_soak_depth=5,
    max_soak_depth=20,
    use_deck_pressure_offset=False,
    exclude_flags=True,
    flag_value=-9.99e-29
)

plot(
    data=data,
    title='Loop Edit',
    x_names=['temp', 'cond', 'filtered_cond', 'corrected_cond'],
    x_bounds={}
)

## Derive TEOS-10
This section shows how the Derive TEOS-10 function uses temperature, conductivity, practical salinity, pressure, latitude, and/or longitude to compute the following thermodynamic parameters using TEOS-10 equations:  

The following table references Python functions from [TEOS-10](https://github.com/TEOS-10/GSW-Python) that correspond to functions in SBE Data Processing.
> Note: Results may differ slightly from SBE Data Processing due to GSW version differences.

| Variable Name                      | GSW-Python function              |  
|------------------------------------|----------------------------------|  
| Absolute Salinity                  | gsw.SA_from_SP                   |
| Absolute Salinity Anomaly          | gsw.deltaSA_from_SP              |
| adiabatic lapse rate               | gsw.adiabatic_lapse_rate_from_CT |
| Conservative Temperature           | gsw.CT_from_t                    |
| Conservative Temperature, freezing | gsw.CT_freezing                  |
| density, TEOS-10                   | gsw.rho[<sup>1</sup>](#teos_fn1)        |
| dynamic enthalpy                   | gsw.dynamic_enthalpy             |
| enthalpy                           | gsw.enthalpy                     |
| entropy                            | gsw.entropy_from_t               |
| gravity                            | gsw.grav                         |
| internal energy                    | gsw.internal_energy              |
| isentropic compressibility         | gsw.kappa                        |
| latent heat of evaporation         | gsw.latentheat_evap_CT           |
| latent heat of melting             | gsw.latentheat_melting           |
| potential temperature              | gsw.pt0_from_t                   |
| Preformed Salinity                 | gsw.Sstar_from_SA                |
| Reference Salinity                 | gsw.SR_from_SP                   |
| saline contraction coefficient     | gsw.beta                         |
| sound speed                        | gsw.sound_speed                  |
| specific volume                    | gsw.specvol                      |
| specific volume anomaly            | gsw.specvol_anom_standard        |
| temperature freezing               | gsw.t_freezing                   |
| thermal expansion coefficient      | gsw.alpha                        |

<span id="teos_fn1"> <sup>1</sup> use gsw.rho with reference pressure for the sigmas

In [ ]:
data['salinity'] = gsw.SP_from_C(
    C=data['corrected_cond'].values,
    t=data['filtered_temp'].values,
    p=data['press'].values
)

data['abs_salinity'] = gsw.SA_from_SP(
    SP=data['salinity'].values,
    p=data['press'].values,
    lon=0,
    lat=0
)

data['density'] = gsw.rho(
    SA=data['abs_salinity'].values,
    CT=data['corrected_cond'].values,
    p=data['press'].values
)

plot(
    data=data,
    title='Derive TEOS-10',
    x_names=['salinity', 'abs_salinity', 'density'],
    x_bounds={0: [2.5, 3.5], 1: [2.5, 3.5], 2: [1000, 1004]}
)

In [ ]:
data['conservative_t'] = gsw.CT_from_t(
    SA=data['abs_salinity'].values,
    t=data['filtered_temp'].values,
    p=data['press'].values
)

data['adiabatic_lapse_rate'] = gsw.adiabatic_lapse_rate_from_CT(
    SA=data['abs_salinity'].values,
    CT=data['conservative_t'].values,
    p=data['press'].values
)

data['dynamic_enthalpy'] = gsw.dynamic_enthalpy(
    SA=data['abs_salinity'].values,
    CT=data['conservative_t'].values,
    p=data['press'].values
)

plot(
    data=data,
    title='Loop Edit',
    x_names=['filtered_temp', 'conservative_t', 'adiabatic_lapse_rate', 'dynamic_enthalpy'],
    x_bounds={}
)

## Wild Edit
This section shows how the Wild Edit function marks wild points in the data by replacing the data value with a badflag value, by default -9.99e-29. Wild Edit’s algorithm requires two passes through the data. The first pass obtains an accurate estimate of the data’s true standard deviation, while the second pass replaces the appropriate data with badflag.  

Wild Edit operates as follows:
1. Compute mean and standard deviation of data in block (specified by Scans per Block) for each selected variable. Temporarily flag values that differ from mean by more than standard deviations specified for pass 1.
2. Recompute mean and standard deviation, excluding temporarily flagged values. Mark values that differ from mean by more than standard deviations specified for pass 2 by replacing data value with badflag.
3. Repeat Steps 1 and 2 for next block of scans.  If last block of data in input file has less than specified number of scans, use data from previous block to fill in block.  

If the data file is particularly corrupted, you may need to run Wild Edit more than once, with different block sizes and number of standard deviations. If the input file has some variables with large values and some with relatively smaller values, it may be necessary to run Wild Edit more than once, varying the value for Keep data within this distance of mean so that it is meaningful for each variable. Better results may also be obtained by increasing Scans per block from 100 to around 500.  

In [ ]:
data['wild_temp'] = p.wild_edit(
    data=data['filtered_temp'],
    flags=data['flag'],
    std_pass_1=2,
    std_pass_2=20, # TODO: this breaks when set to 3 or lower
    scans_per_block=100,
    distance_to_mean=0,
    exclude_bad_flags=True
)

plot(
    data=data,
    title='Wild Edit',
    x_names=['filtered_temp', 'wild_temp'],
    x_bounds={}
)

## Window Filter
The Window Filter function gives four types of window filters and a median filter that can be used in the smoothing of data:  
- Window filters calculate a weighted average of data values about a center point and replace the data value at the center point with this average.  
- The median filter calculates a median for data values about a center point and replaces the data value at the center point with the median 

### Descriptions and Formulas
Shape and length define filter windows:  
- Window Filter gives four window shapes: boxcar, cosine, triangle, and Gaussian.  
- The minimum window length is 1 scan, and the maximum is 511 scans. Window length must be an odd number, so that the window has a center point. If a window length is specified as an even number, Window Filter automatically adds 1 to make the length odd.  

The window filter calculates a weighted average of data values about a center point, using the following transfer function:  

&emsp; $y(n) = \displaystyle\sum_{k=-L/2}^{L/2}w(k)x(n-k)$  

The window filtering process is similar for all filter types:  
1. Filter weights are calculated (see the equations below).  
2. Filter weights are normalized to sum to 1.
    - When a bad data point is encountered (scan marked with badflag if exclude scans marked bad was selected or data value marked with badflag), the weights are renormalized, excluding the filter element that would operate on the bad data point.  

In the following equations:  
&emsp; $L = window\ length\ in\ scans\ (must\ be\ odd)$  
&emsp; $n = window\ index,\ -\frac{L-1}{2}..\frac{L-1}{2}$  
&emsp; $w(n) = set\ of\ window\ weights$  

#### Boxcar Filter
&emsp; $w(n) = \frac{1}{L}$  

#### Cosine Filter
&emsp; $w(n) = cos(\frac{n\pi}{L+1})$

#### Triangle Filter
&emsp; $w(n) = 1-\frac{|2n|}{L+1}$

#### Gaussian Filter
&emsp; $phase = \frac{offset}{sample\ interval}$  

&emsp; $scale = log(2)(2\frac{sample\ rate}{half\ width\ (scans)})^2$  

&emsp; $w(n) = e^{(n-phase)^2 * scale}$  

The Gaussian window has parameters of halfwidth (in scans) and offset (in time), in addition to window length (in scans). These extra parameters allow data to be filtered and shifted in time in one operation. Halfwidth determines the width of the Gaussian curve. A window length of 9 and halfwidth of 4 produces a set of filter weights that fills the window. A window length of 17 and halfwidth of 4 produces a set of filter weights that fills only half the window. If the filter weights do not fill the window, the offset parameter may be used to shift the weights within the window without clipping the edge of the Gaussian curve.  

#### Median Filter
The median filter is not a smoothing filter in the same sense as the window filters described above. Median filtering is most useful in spike removal. A median value is determined for a specified window, and the data value at the window’s center point is replaced by the median value.  

In [ ]:
data['boxcar_temp'] = p.window_filter(
    data_in=data['temp'],
    flags=data['flag'],
    window_type=p.WindowFilterType.BOXCAR,
    window_width=21,
    sample_interval=0.25,
)

plot(
    data=data,
    title='Boxcar Window Filter',
    x_names=['temp', 'boxcar_temp'],
    x_bounds={}
)